In [1]:
# Conversion done using UniProKB
import sentencepiece as spm
import pandas as pd
from Bio import SeqIO

In [2]:
conv1 = pd.read_csv('g1.tsv', sep='\t')
conv1 = conv1.rename(columns={"yourlist:M201908296746803381A1F0E0DB47453E0216320D99E33CI":"Gene"})

In [3]:
conv1 = conv1[['Gene', 'Sequence']]
g1 = conv1['Gene'].to_list()
p1 = conv1['Sequence'].to_list()
conv_dict_1 = dict(zip(g1, p1))

In [4]:
keys = list(conv_dict_1.keys())

In [5]:
df1 = pd.read_csv('1.txt', sep='\t')
gene_list_1 = df1['Gene1'].to_list()
gene_list_2 = df1['Gene2'].to_list()
assert len(gene_list_1) == len(gene_list_2)
print(len(gene_list_1))

7865


In [6]:
def fix_keys(keys, d, ref_list):
    for k in keys:
        k_array = k.split(',')
        if len(k_array) > 1:
            for i in range(len(k_array)):
                if k_array[i] in ref_list:
                    d[k_array[i]] = d[k]
            del d[k]

In [7]:
fix_keys(keys, conv_dict_1, gene_list_1)

In [8]:
conv_dict_1['SH2D2A']

'MEFPLAQICPQGSHEAPIPTFSTFQITDMTRRSCQNLGYTAASPQAPEAASNTGNAERAEEVPGEGSLFLQAETRAWFQKTQAHWLLQHGAAPAWFHGFITRREAERLLEPKPQGCYLVRFSESAVTFVLTYRSRTCCRHFLLAQLRDGRHVVLGEDSAHARLQDLLLHYTAHPLSPYGETLTEPLARQTPEPAGLSLRTEESNFGSKSQDPNPQYSPIIKQGQAPVPMQKEGAGEKEPSQLLRPKPPIPAKPQLPPEVYTIPVPRHRPAPRPKPSNPIYNEPDEPIAFYAMGRGSPGEAPSNIYVEVEDEGLPATLGHPVLRKSWSRPVPGGQNTGGSQLHSENSVIGQGPPLPHQPPPAWRHTLPHNLSRQVLQDRGQAWLPLGPPQ'

In [9]:
conv2 = pd.read_csv('g2.tsv', sep='\t')
conv2 = conv2.rename(columns={"yourlist:M201908296746803381A1F0E0DB47453E0216320D99E36A3":"Gene"})

In [10]:
conv2 = conv2[['Gene', 'Sequence']]
g2 = conv2['Gene'].to_list()
p2 = conv2['Sequence'].to_list()
conv_dict_2 = dict(zip(g2, p2))

In [11]:
keys2 = list(conv_dict_2.keys())

In [12]:
fix_keys(keys2, conv_dict_2, gene_list_2)

In [13]:
conv_dict_2['SNRPD2']

'MSLLNKPKSEMTPEELQKREEEEFNTGPLSVLTQSVKNNTQVLINCRNNKKLLGRVKAFDRHCNMVLENVKEMWTEVPKSGKGKKKSKPVNKDRYISKMFLRGDSVIVVLRNPLIAGK'

In [14]:
conv_dict_2['HIST1H3E'] = 'MQIFVKTLTGKTITLEVEPSDTIENVKAKIQDKEGIPPDQQRLIFAGKQLEDGRTLSDYNIQKESTLHLVLRLRGC'

In [15]:
conv_dict_1['HIST2H3A'] = 'KSTELLIRKLPFQRLVREIAQDFKTDLRFQSSAVMALQEASEAYLVGLFEDTNLCAIHAKRVTIMPKDIQLARRIRGERA'

In [16]:
conv_dict_2['XIST'] = 'GGCGCAUCGGCGCC'

In [17]:
with open('protein_positive_a.txt', 'w') as f1, \
    open('protein_positive_b.txt', 'w') as f2:
    for g1,g2 in zip(gene_list_1, gene_list_2):
        f1.write('_' + conv_dict_1[g1] + '\n')
        f2.write('_' + conv_dict_2[g2] + '\n')

In [18]:
neg = pd.read_csv('human_all.tsv', sep='\t')
neg = neg[['Sequence', 'Subcellular location [CC]']]
neg = neg.rename(columns={"Subcellular location [CC]":"Location"})
neg = neg[neg['Location'].isnull() == False]

In [19]:
#Establish negative interaction set
neg_size = len(gene_list_1)
with open('protein_negative_a.txt', 'w') as f1, \
    open('protein_negative_b.txt', 'w') as f2:
    for _ in range(neg_size):
        sampled_data = neg.sample(n=2)
        sample_loc = sampled_data['Location'].to_list()
        #randomly sample from different subcellular locations
        while sample_loc[0] == sample_loc[1]:
            sampled_data = neg.sample(n=2)
            sample_loc = sampled_data['Location'].to_list()
        seq = sampled_data['Sequence'].to_list()
        f1.write('_' + seq[0] + '\n')
        f2.write('_' + seq[1] + '\n')

In [20]:
filenames = ['protein_positive_a.txt', 'protein_negative_a.txt', 'protein_positive_b.txt', 'protein_negative_b.txt']
with open('all_proteins.txt', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                if line[0] == '_':
                    outfile.write(line[1:])
                else:
                    outfile.write(line)

In [17]:
# Train a sentence piece model to segment proteins
spm.SentencePieceTrainer.Train('--input=/Users/Kbutterstrap/Desktop/PPI_Code/all_proteins.txt --model_prefix=/Users/Kbutterstrap/Desktop/PPI_Code/seg')

True

In [21]:
model_path = '/Users/Kbutterstrap/Desktop/PPI_Code/seg.model'
sp = spm.SentencePieceProcessor()
sp.Load(model_path)

True

In [22]:
filenames = ['protein_positive_a.txt', 'protein_negative_a.txt']
with open('proteins_a.txt', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

In [23]:
filenames = ['protein_positive_b.txt', 'protein_negative_b.txt']
with open('proteins_b.txt', 'w') as outfile:
    for fname in filenames:
        with open(fname) as infile:
            for line in infile:
                outfile.write(line)

In [31]:
with open('tokenized_a.txt', 'w') as f, \
    open('proteins_a.txt', 'r') as f_a:
    start = 0
    for lines in f_a:
        #first line of the file
        if start == 0:
            current_protein = lines[1:]
        #first line of another protein sequence
        if (lines[0] == '_') & (start != 0):
            token_list = sp.EncodeAsPieces(current_protein)
            for t in token_list:
                if token_list.index(t) == 0:
                    f.write('\n' + '_' + t + ' ')
                else:
                    f.write(t + ' ')
            current_protein = lines[1:]
        #within the range of one protein sequence
        else:
            current_protein += lines
            start += 1

In [32]:
with open('tokenized_b.txt', 'w') as f, \
    open('proteins_b.txt', 'r') as f_a:
    start = 0
    for lines in f_a:
        #first line of the file
        if start == 0:
            current_protein = lines[1:]
        #first line of another protein sequence
        if (lines[0] == '_') & (start != 0):
            token_list = sp.EncodeAsPieces(current_protein)
            for t in token_list:
                if token_list.index(t) == 0:
                    f.write('\n' + '_' + t + ' ')
                else:
                    f.write(t + ' ')
            current_protein = lines[1:]
        #within the range of one protein sequence
        else:
            current_protein += lines
            start += 1